In [1]:
import sys
# sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
# sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read all files

In [7]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tk = th.Thicket.from_caliperreader(glob("BitonicSort/cali_files/*.cali"))

(2/2) Creating Thicket: 100%|██████████| 253/253 [00:01<00:00, 192.21it/s]


View Calltree

In [8]:
print(tk.tree(metric_column="Max time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

0.001 MPI_Comm_dup
0.000 MPI_Finalize
0.000 MPI_Finalized
0.000 MPI_Initialized
2.003 main
├─ 0.002 MPI_Barrier
├─ 0.021 MPI_Reduce
├─ 0.025 comm
│  └─ 0.025 comm_large
│     ├─ 0.006 MPI_Gather
│     └─ 0.024 MPI_Scatter
├─ 1.942 comp
│  └─ 1.942 comp_large
│     └─ 0.018 MPI_Sendrecv
├─ 0.015 correctness_check
└─ 0.018 data_init_runtime

Legend (Metric: Max time/rank Min: 0.00 Max: 2.00 indices: {'profile': 20360507})
█ 1.80 - 2.00
█ 1.40 - 1.80
█ 1.00 - 1.40
█ 0.60 - 1.00
█ 0.20 - 0.60
█ 0.00 - 0.20

name User code    ◀  Only in left graph    ▶  Only in right graph



: 

Group Performance data by `matrix_size` in the Thicket metadata table.

In [103]:
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "num_procs", "input_size", "input_type"]).sort_index()

tk.dataframe.head(20)

profile  \
node                                 num_procs input_size input_type                             
{'name': 'main', 'type': 'function'} 2         65536      Random                    3669930900   
                                                          Reverse Sorted            2415705447   
                                                          Sorted                     300411230   
                                                          Sorted with 1% perturbed   972226623   
                                               262144     Random                    3402972926   
                                                          Reverse Sorted            1198010761   
                                                          Sorted                    2108853330   
                                                          Sorted with 1% perturbed  3003965217   
                                               1048576    Random                     440470322   
                                                          Reverse Sorted            3816414986   
                                                          Sorted                    1536058806   
                                                          Sorted with 1% perturbed   261843834   
                                               4194304    Random                     603037390   
                                                          Reverse Sorted            2852939213   
                                                          Sorted                    3523986067   
                                                          Sorted with 1% perturbed   884934137   
                                               16777216   Random                     477269477   
                                                          Reverse Sorted            3036698840   
                                                          Sorted                     875396164   
                                                          Sorted with 1% perturbed   655752124   

                                                                                    nid  \
node                                 num_procs input_size input_type                      
{'name': 'main', 'type': 'function'} 2         65536      Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               262144     Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               1048576    Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               4194304    Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                                                          Sorted with 1% perturbed  1.0   
                                               16777216   Random                    1.0   
                                                          Reverse Sorted            1.0   
                                                          Sorted                    1.0   
                   

In [98]:
# Define common variables
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
input_sizes = [2**i for i in range(16, 30, 2)]
input_types = ["Random", "Sorted", "Reverse Sorted", "Sorted with 1% perturbed"]

In [91]:
# Change font size for all plots
plt.rcParams.update({"font.size": 20})

In [122]:
def create_scaling_plots(df):
    input_sizes = df.index.get_level_values('input_size').unique()
    num_procs = df.index.get_level_values('num_procs').unique()
    input_types = df.index.get_level_values('input_type').unique()
    nodes = ['comp_large', 'comm', 'main']

    for node in nodes:
        for size in input_sizes:
            fig, ax = plt.subplots(figsize=(10, 7))
            ax.set_title(f'Strong Scaling - {node} - Input Size {size}', fontsize=16)
            size_data = df[df.index.get_level_values('input_size') == size]
            for input_type in input_types:
                input_data = size_data[size_data.index.get_level_values('input_type') == input_type]
                tempList = [n.frame.get("name") for n in input_data.index.get_level_values("node").values]
                nodeLoc = input_data.index.get_level_values("node").values[tempList.index(node)]
                data = input_data.loc[nodeLoc]
                
                ax.plot(num_procs, data['Total time'], label=input_type)
            ax.set_xscale('log', base=2)
            ax.set_xlabel('Number of Processes')
            ax.set_ylabel('Total Time (s)')
            ax.legend(title='Input Type')
            plt.savefig(f'samplesort_presentation_graphs/strong_scaling_{node}_input_size_{size}.png')
            plt.close()

        for input_type in input_types:
            fig, ax = plt.subplots(figsize=(10, 7))
            ax.set_title(f'Strong Scaling Speedup - {node} - {input_type}', fontsize=16)
            input_data = df[df.index.get_level_values('input_type') == input_type]
            for size in input_sizes:
                size_data = input_data[input_data.index.get_level_values('input_size') == size]
                tempList = [n.frame.get("name") for n in size_data.index.get_level_values("node").values]
                nodeLoc = size_data.index.get_level_values("node").values[tempList.index(node)]
                data = size_data.loc[nodeLoc]

                speedup = data['Total time'].iloc[0] / data['Total time']
                ax.plot(num_procs, speedup, label=f'Size {size}')
            ax.set_xscale('log', base=2)
            ax.set_xlabel('Number of Processes')
            ax.set_ylabel('Speedup')
            ax.legend(title='Input Size')
            plt.savefig(f'samplesort_presentation_graphs/strong_scaling_speedup_{node}_{input_type}.png')
            plt.close()

        for input_type in input_types:
            fig, ax = plt.subplots(figsize=(10, 7))
            ax.set_title(f'Weak Scaling - {node} - {input_type}', fontsize=16)
            input_data = df[df.index.get_level_values('input_type') == input_type]
            for procs in num_procs:
                proc_data = input_data[input_data.index.get_level_values('num_procs') == procs]
                tempList = [n.frame.get("name") for n in proc_data.index.get_level_values("node").values]
                nodeLoc = proc_data.index.get_level_values("node").values[tempList.index(node)]
                data = proc_data.loc[nodeLoc]

                ax.plot(input_sizes, data['Total time'], label=f'{procs} Processes')
            ax.set_xscale('log', base=2)
            ax.set_xlabel('Input Size')
            ax.set_ylabel('Total Time (s)')
            ax.legend(title='Number of Processes')
            plt.savefig(f'samplesort_presentation_graphs/weak_scaling_{node}_{input_type}.png')
            plt.close()

create_scaling_plots(tk.dataframe)

Index([65536, 262144, 1048576, 4194304, 16777216, 67108864, 268435456], dtype='int64', name='input_size')
